# Using the AVID Public APIs

## Background 

AVID houses two types of records: *Records* and *Vulnerabilities*. A vulnerability (vuln) is high-level evidence of an AI failure mode. A report is one example of a particular vulnerability occurring, supported by qualitative or quantitative evaluation. You can think of a report as an instance of a vulnerability.

As an example, we'll look at a vulnerability below about gender bias in a particular language model, `xlm-roberta-base`. That vuln is associated with two reports, each of which measured gender bias by a different method.

Only reports can be submitted to the API; vulnerabilities cannot be submitted. After a report is submitted, it becomes a "draft" and enters the AVID editorial queue. In the editorial process, human editors review and validate the report and determine whether it belongs to an existing vulnerability or represents a new vulnerability. Reports are published after successfully passing through the editorial process. 

In this notebook, we'll walk through the process of submitting a new report and checking on its status. We'll also cover how to retrieve published items from the database.

## Public API Documentation
| Method | HTTP request | Description |
|------------- | ------------- | -------------|
| [**healthCheck**](#healthCheck) | **GET** / | health check |
| [**submitObjectByObjectType**](#submitObjectByObjectType) | **POST** /submit/object/{obj_type} | submit an object for review by object type |
| [**retrieveStatusByUUID**](#retrieveStatusByUUID) | **GET** /review/{uuid}/status | get editorial status of submitted object |
| [**listObjects**](#listObjects) | **GET** /objects | get published avid objects |
| [**getObjectByAvidID**](#getObjectByAvidID) | **GET** /object/{avid_id} | get published object by avid id |
| [**listObjectsByObjectType**](#listObjectsByObjectType) | **GET** /objects/{obj_type} | get published objects by object type |


In [ ]:
import json
from os import environ
import requests

In [ ]:
base = "" #replace with the URL provided by AVID
api_key = environ.get('AVID_API_KEY')
headers =  {"Authorization":api_key}

# Basic API check
This is a quick check to make sure your API key is working and you're able to contact the endpoint.  
It should give the response "meow".

<a name="healthCheck"></a>
### **healthCheck**
> String healthCheck()

health check

    This is an endpoint for sanity checks. 

#### Parameters
This endpoint does not need any parameter.

#### Return type

**String**

#### HTTP request headers

- **Content-Type**: Not defined
- **Accept**: text/plain


In [ ]:
url = base
response = requests.get(url, headers=headers)
print(response, response.text)

# Submitting reports

## How to format your report

Reports should conform to the AVID report data model, which is documented [here](https://avidml.org/avidtools/reference/report.html). The endpoint does not perform validation against this data model; it accepts any valid JSON. A validation endpoint will be available in the future.

Different evaluation pipelines can be integrated with `avidtools` to support creating AVID reports within that pipeline. As an illustrative example, [check out](https://avidml.gitbook.io/doc/developer-tools/python-sdk/integrations/garak) our documentation summarizing an existing integration with the LLM Vulnerability Scanner garak. You can see the source code for this integration [here](https://github.com/leondz/garak/blob/main/analyze/report_avid.py). To this end, you can also check out the source code for our [Hugging Face space](https://huggingface.co/spaces/avid-ml/bias-detection).

## Submit an object

Here we'll submit a draft report generated by [this HuggingFace space](https://huggingface.co/spaces/avid-ml/bias-detection), which reports gender bias in a language models. The HuggingFace space produces the report as a string, so we'll start by loading that into a json object.

<a name="submitObjectByObjectType"></a>
### **submitObjectByObjectType**
> String submitObjectByObjectType(obj\_type, AvidObjectData)

submit an object for review by object type

    public API to submit an object for review. upon success,  an editorial entry will be created in the backend, and  the api returns the uuid for that entry. it could be used to track the status. 

#### Parameters

|Name | Type | Description  | Notes |
|------------- | ------------- | ------------- | -------------|
| **obj\_type** | **AvidObjectType**| type of object to list | [default to null] [enum: report, vulnerability] |
| **AvidObjectData** | **AvidObjectData**| data of an object to be submitted for review | [optional] |

#### Return type

**String**

#### HTTP request headers

- **Content-Type**: application/json
- **Accept**: application/json, text/plain

In [ ]:
new_report = json.loads('''{
  "data_type": "AVID",
  "data_version": null,
  "metadata": null,
  "affects": {
    "developer": [],
    "deployer": [
      "Hugging Face"
    ],
    "artifacts": [
      {
        "type": "Model",
        "name": "bert-base-cased"
      }
    ]
  },
  "problemtype": {
    "classof": "LLM Evaluation",
    "type": "Detection",
    "description": {
      "lang": "eng",
      "value": "Profession bias reinforcing gender stereotypes found in bert-base-cased, as measured on the Winobias dataset"
    }
  },
  "metrics": [
    {
      "name": "Winobias",
      "detection_method": {
        "type": "Significance Test",
        "name": "One-sample Z-test"
      },
      "results": {
        "feature": [
          "gender"
        ],
        "stat": [
          19.91
        ],
        "pvalue": [
          0
        ]
      }
    }
  ],
  "references": [
    {
      "label": "Winograd-schema dataset for detecting gender bias",
      "url": "https://uclanlp.github.io/corefBias/overview"
    },
    {
      "label": "bert-base-cased on Hugging Face",
      "url": "https://huggingface.co/bert-base-cased"
    }
  ],
  "description": {
    "lang": "eng",
    "value": "Filling in pronouns in sentences tagged with professions using bert-base-cased were found to be significantly biased on the Winobias dataset."
  },
  "impact": {
    "avid": {
      "risk_domain": [
        "Ethics"
      ],
      "sep_view": [
        "E0101: Group fairness"
      ],
      "lifecycle_view": [
        "L05: Evaluation"
      ],
      "taxonomy_version": "0.2"
    }
  },
  "credit": null,
  "reported_date": "2023-07-12"
}''', parse_float=str)


In [ ]:
url = base + "/submit"
response = requests.post(url, json=new_report, headers=headers)
uuid = response.json()
print(response, uuid)

The endpoint returns the UUID, a unique identifier for the submitted entry. The UUID can be used to track the status of a submission, as shown below.

## Retrieve the editorial status of a submitted report object

Now we'll check on the status of the report we just submitted above. The API should return the status as "draft."

<a name="retreiveStatusByUUID"></a>
### **retreiveStatusbyUUID**
> string retreiveStatusByUUID(UUID)

get editorial status for a submitted object

    public API to get the current editorial status of a submitted object.

#### Parameters
This endpoint uses the UUID in the URL as a parameter.

#### Return type

**string**

#### HTTP request headers

- **Content-Type**: Not defined
- **Accept**: application/json, text/plain

In [ ]:
url = base + f"/review/{uuid}/status"
response = requests.get(url, headers=headers)
print(response, response.json())

# Retrieving reports and vulnerabilities from the database

## Get all published objects

Published objects include both reports and vulnerabilities. Run the cell below to retrieve them all and take a look at a few of them.

<a name="listObjects"></a>
### **listObjects**
> List listObjects()

get published avid objects

    public API to list published avid objects.

#### Parameters
This endpoint does not need any parameter.

#### Return type

**List**

#### HTTP request headers

- **Content-Type**: Not defined
- **Accept**: application/json, text/plain

In [ ]:
url = base + "/objects"
response = requests.get(url, headers=headers)
print(response)
for r in response.json()[:3]:
    print(json.dumps(r, indent=2))
    print("##########################################")

## Get published objects by AVID ID

Published reports and vulnerabilities have IDs of the form `AVID-2022-V001` for vulnerabilities and `AVID-2022-R0001` for reports. You can retrieve published objects by these IDs. 


<a name="getObjectByAvidID"></a>
### **getObjectByAvidID**
> AvidObjectFull getObjectByAvidID(avid\_id)

get published object by avid id

    public API to get a published avid object by its published id.

#### Parameters

|Name | Type | Description  | Notes |
|------------- | ------------- | ------------- | -------------|
| **avid\_id** | **AvidID**| published avid object id | [default to null] |

#### Return type

**AvidObjectFull**

#### HTTP request headers

- **Content-Type**: Not defined
- **Accept**: application/json, text/plain

In [ ]:
url = base + "/object/AVID-2022-R0004"
response = requests.get(url, headers=headers)
print(response)
print(json.dumps(response.json(), indent=2))

Above, we retrieved a report. The same type of request works for vulns too. In that case, just supply a vuln ID instead of a report ID. 

## Get published objects by object type

When retrieving all published objects, you can limit your results to just reports, or just vulns, as follows.

<a name="listObjectsByObjectType"></a>
### **listObjectsByObjectType**
> List listObjectsByObjectType(obj\_type)

get published objects by object type

    public API to list published objects by object_type.  convenient in_path filter of {/objects}.  

#### Parameters

|Name | Type | Description  | Notes |
|------------- | ------------- | ------------- | -------------|
| **obj\_type** | **AvidObjectType** | type of object to list | [default to null] [enum: report, vulnerability] |

#### Return type

**List**

#### HTTP request headers

- **Content-Type**: Not defined
- **Accept**: application/json, text/plain

In [ ]:
url = base + "/objects/report"
response = requests.get(url, headers=headers)
print(response)
print(response.json()[0])   # print the first one

In [ ]:
url = base + "/objects/vulnerability"
response = requests.get(url, headers=headers)
print(response)
print(response.json()[0])   # print the first one